In [1]:
import sys
# sys.path.append(r"C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool") 
from MatchSys import config


from MatchSys import MatchSys
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'data\model.bin',
        vector_similarity_rate=0.7,
        most_similar_number=10,
        vector_match_times=5,
        need_ltp=False,
        search_adapters_name_list=['MatchSys.search.TextSearch','MatchSys.search.FaissSearch'],
        signal_file_size=10000
    )

Starting initialization of ModelBase
Starting initialization of MatchSys
Use pytorch device: cuda


In [2]:
ms.get_response('我喜欢你')

----
[[3.1761438e-10 3.1761438e-10 5.5545631e+01 5.5545631e+01 1.0104325e+02]]
----
[[1724673099296804864 1724678250967470082 1724673119697899522
  1724678274178748418 1724673187742093315]]


ValueError: operands could not be broadcast together with shapes (2,) (1,5) 

In [ ]:
from MatchSys.trainer.base_qa_trainer import QATrainer

trainer = QATrainer(ms)

import pandas as pd
map = {}
data = pd.read_excel(r"data\FixedReply\Test-1K.xlsx",header=None, sheet_name=0)
for index,row in data.iterrows():
    row[0] = str(row[0]).strip()
    row[1] = str(row[1]).strip()
    if row[0] in map:
        map[row[0]].append(row[1])
    else:
        map[row[0]] = [row[1]]
  
        
trainer.train(map)

In [ ]:
from MatchSys.trainer.base_chat_list_trainer import ChatListTrainer

trainer = ChatListTrainer(ms)
data = [
    [
        '可战争在哪儿？现在全球一处热点都没有，应该是历史上最和平的年代了。',
        '没有。',
        '那你的生活是一种偶然，世界有这么多变幻莫测的因素，你的人生却没什么变故。',
        '大部分人都是这样嘛。',
        '那大部分人的人生都是偶然。',
        '可......多少代人都是这么平淡地过来的。',
        '都是偶然。',
        '得承认今天我的理解力太差了，您这岂不是说......',
        '是的，整个人类历史也是偶然，从石器时代到今天，都没什么重大变故，真幸运。但既然是幸运，总有结束的一天；现在我告诉你，结束了，做好思想准备吧。',
    ]
]
        
trainer.train(data)

In [ ]:
len(ms.search_adapters[0].model.dv.index_to_key)

In [ ]:
while True:
    message = input('>')
    if message == 'exit':
        break
    else:
        print(ms.get_response(message))

In [1]:
import torch
torch.cuda.is_available()


True